For RetinaNet, the parameters must be in the form of:
path/to/img.jpg,x1,y1,x2,y2,class_name
<hr>
class_name specified above must be a class that exists in a CSV file in the format:
str name, int classId

In [7]:
#import bounding box code from phase 2
%pylab inline
import csv
import os, sys
import xml.etree.ElementTree as ET
import matplotlib.patches as patches
import matplotlib.pyplot as pl
import time
import pylab as pl
from IPython import display


Populating the interactive namespace from numpy and matplotlib


/Users/stormking/opt/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['display']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Reading the Data
Read the data from the xml file and then convert to csv files necessary for RetinaNet.

In [10]:
#Remove Data and Class files if they exist
if os.path.exists("RetinaNetTrainData.csv"):
    os.remove("RetinaNetTrainData.csv")
if os.path.exists("RetinaNetTestData.csv"):
    os.remove("RetinaNetTestData.csv")
if os.path.exists("RetinaNetClasses.csv"):
    os.remove("RetinaNetClasses.csv")
    
#Generate the data and write to file
i = 1
for file_index in range(5,8):
    if file_index == 0:
        file_to_parse = 'CityCam/164/164-20160223-09/'
    elif file_index == 1:
        file_to_parse = 'CityCam/166/166-20160214-11/'
    elif file_index == 2:
        file_to_parse = 'CityCam/170/170-20160425-18/'
    elif file_index == 3:
        file_to_parse = 'CityCam/173/173-20160429-07/'
    elif file_index == 4:
        file_to_parse = 'CityCam/181/181-20151224-15/'
    elif file_index == 5:
        file_to_parse = 'CityCam/410/410-20160429-07/'
    elif file_index == 6:
        file_to_parse = 'CityCam/495/495-20160429-07/'
    elif file_index == 7:
        file_to_parse = 'CityCam/928/928-20160429-07/'
    
    while i<201:
        j = str(i)
        full_jpg_path = ''
        if i < 10:
            tree = ET.parse(file_to_parse +'00000'+j+'.xml')
            im = imread(file_to_parse +'00000'+j+'.jpg')
            full_jpg_path = file_to_parse +'00000'+j+'.jpg'
        elif i < 100:
            tree = ET.parse(file_to_parse +'0000'+j+'.xml')
            im = imread(file_to_parse +'0000'+j+'.jpg')
            full_jpg_path = file_to_parse +'0000'+j+'.jpg'
        else:
            tree = ET.parse(file_to_parse +'000'+j+'.xml')
            im = imread(file_to_parse +'000'+j+'.jpg')
            full_jpg_path = file_to_parse +'000'+j+'.jpg'
            
        root = tree.getroot()
        bounding_boxes = []
        classes = []
        
        for child in root:
            #Store vehicle and passengers tags
            if child.tag == str("vehicle") or child.tag == str("passengers"):
                classes.append(child.tag)
            #Store bounding box locations
            for x in child:
                bounding_box_child = []
                for y in x:
                    bounding_box_child.append(y.text)
                if len(bounding_box_child) == 4:
                    bounding_boxes.append(bounding_box_child)
        #print("Info for Folder {0}".format(file_index + 1))
        #print("File Path: {0}".format(full_jpg_path))
        #print("Bounding Boxes: {0}".format(bounding_boxes))
        #print("Annotations: {0}".format(classes))
        
        #Generate train data from 150 examples from this file
        if i <= 151:
            for entry in range(len(bounding_boxes)):
                path_for_google_drive = '/content/drive/My Drive/' + full_jpg_path
                xMin = max(0, int(bounding_boxes[entry][1]))
                yMin = max(0, int(bounding_boxes[entry][3]))
                xMax = min(352, int(bounding_boxes[entry][0]))
                yMax = min(240, int(bounding_boxes[entry][2]))
                row = [path_for_google_drive, xMin, yMin, xMax, yMax, classes[entry]]
                #write rows to a csv file
                with open('RetinaNetTrainData.csv', 'a') as csvFile:
                    writer = csv.writer(csvFile)
                    writer.writerow(row)
                csvFile.close()
        #Generate test data from 50 examples from this file
        else:
            for entry in range(len(bounding_boxes)):
                path_for_google_drive = '/content/drive/My Drive/' + full_jpg_path
                xMin = max(0, int(bounding_boxes[entry][1]))
                yMin = max(0, int(bounding_boxes[entry][3]))
                xMax = min(352, int(bounding_boxes[entry][0]))
                yMax = min(240, int(bounding_boxes[entry][2]))
                row = [path_for_google_drive, xMin, yMin, xMax, yMax, classes[entry]]
                #write rows to a csv file
                with open('RetinaNetTestData.csv', 'a') as csvFile:
                    writer = csv.writer(csvFile)
                    writer.writerow(row)
                csvFile.close()
            
        i = i+1
    i=1
    
#Generate class csv file
with open('RetinaNetClasses.csv', 'a') as csvFile:
    writer = csv.writer(csvFile)
    row1 = ['vehicle', 0]
    row2 = ['passengers', 1]
    writer.writerow(row1)
    writer.writerow(row2)
csvFile.close()
        